In [1]:
import openai
from utils import chat

# load and set our key
try:
    openai.api_key = open("key.txt", "r").read().strip("\n")
except FileNotFoundError:
    openai.api_key = input("Please enter your OpenAI API key: ")
    with open("key.txt", "w") as f:
        f.write(openai.api_key)

In [2]:
chat(customer_is='gpt')

Customer:  Hi. I am not sure about how to invest. Could you help me?
{'age': False, 'income': False, 'risk appetite': False}
